In [1]:
import pandas as pd
import subprocess
import os
import datetime as dt
import numpy as np

In [2]:
cases_dict = {
    0: {"n": [2], "r": 10, "k": 2},
    1: {"n": [2,4,8,16,32], "r": 10, "k": 2},
    2: {"n": [2,4,8,16,32,64], "r": 5, "k": 1},
    3: {"n": [2,4,8,16,32,64], "r": 3, "k": 0}
}
file_path = r'C:\Users\anabe\Desktop\be\ufrj\sistemas_distribuidos\sistemas_distribuidos_tp3\log.txt'

In [3]:
for case in cases_dict:
    n, r, k = cases_dict[case]["n"], cases_dict[case]["r"], cases_dict[case]["k"]
    for i in n:
        print(f"STARTING CASE {case} (N {i}) (R {r}) (K {k})")
        df_final = pd.DataFrame()
        try:
            os.remove(file_path)
            print(f"File '{file_path}' deleted successfully.")
        except FileNotFoundError:
            print(f"File '{file_path}' not found.")
        except Exception as e:
            print(f"An error occurred while trying to delete the file: {e}")

        with open("log.txt", "w") as file:
            # Write the first line to the file
            file.write("Thread ID, Time, Message Type\n")  # \n inserts a line break
        try:
            # server_process = subprocess.run(["wsl", "./coordenador"], capture_output=True, text=True, check=True)
            process = subprocess.run(["wsl", "./gerenciador_processos", str(i), str(r), str(k)], capture_output=True, text=True, check=True)
            # server_process.terminate()
            if process.stderr:
                print("C script errors:")
                print(process.stderr)
        except subprocess.CalledProcessError as e:
            print(f"Error running C script: {e}")
        except FileNotFoundError:
            print("C script executable not found.")
    
        df = pd.read_csv(file_path)
        df['n'] = i
        df['r'] = r
        df['k'] = k
        df['case'] = case
        df_final = pd.concat([df_final, df])
        df_final.to_csv(f'results/case_{case}_{n}_{r}_{k}.csv', index=False)

STARTING CASE 0 (N 2) (R 10) (K 2)
File 'C:\Users\anabe\Desktop\be\ufrj\sistemas_distribuidos\sistemas_distribuidos_tp3\log.txt' deleted successfully.


C script errors:
Connection failed: Connection refused
Connection failed: Connection refused

STARTING CASE 1 (N 2) (R 10) (K 2)
File 'C:\Users\anabe\Desktop\be\ufrj\sistemas_distribuidos\sistemas_distribuidos_tp3\log.txt' deleted successfully.
C script errors:
Connection failed: Connection refused
Connection failed: Connection refused

STARTING CASE 1 (N 4) (R 10) (K 2)
File 'C:\Users\anabe\Desktop\be\ufrj\sistemas_distribuidos\sistemas_distribuidos_tp3\log.txt' deleted successfully.
C script errors:
Connection failed: Connection refused
Connection failed: Connection refused
Connection failed: Connection refused
Connection failed: Connection refused

STARTING CASE 1 (N 8) (R 10) (K 2)
File 'C:\Users\anabe\Desktop\be\ufrj\sistemas_distribuidos\sistemas_distribuidos_tp3\log.txt' deleted successfully.
C script errors:
Connection failed: Connection refused
Connection failed: Connection refused
Connection failed: Connection refused
Connection failed: Connection refused
Connection failed: C

In [4]:
df_total = pd.DataFrame()
for case in cases_dict:
    df = pd.read_csv(f'case_{case}.csv')
    df.columns = list(map(lambda x: x.strip(), df.columns))
    df['Thread ID'] = df['Thread ID'].apply(lambda x: int(x.split(': ')[1]))
    df['Time'] = df['Time'].apply(lambda x: dt.datetime.strptime(x.split(': ')[1].split(' ')[0], '%H:%M:%S:%f').time())
    df = df.drop(columns=["Message Type"])
    df_total = pd.concat([df_total, df])

In [5]:
unique_combinations = np.array(df_total[['n', 'k', 'r', 'case']].drop_duplicates())
unique_combinations

array([[  2,   2,  10,   0],
       [  2,   2,  10,   1],
       [  4,   2,  10,   1],
       [  8,   2,  10,   1],
       [ 16,   2,  10,   1],
       [ 32,   2,  10,   1],
       [  2,   1,   5,   2],
       [  4,   1,   5,   2],
       [  8,   1,   5,   2],
       [ 16,   1,   5,   2],
       [ 32,   1,   5,   2],
       [ 64,   1,   5,   2],
       [  2,   0,   3,   3],
       [  4,   0,   3,   3],
       [  8,   0,   3,   3],
       [ 16,   0,   3,   3],
       [ 32,   0,   3,   3],
       [ 64,   0,   3,   3],
       [128,   0,   3,   3]], dtype=int64)

In [6]:
for comb in unique_combinations[:3]:
    n, k, r, case = comb[0], comb[1], comb[2], comb[3]
    print(f"Combinação N = {n}; K = {k}; R = {r}; Case = {case}")
    df_prov = df_total.loc[(df_total['n'] == n) & (df_total['k'] == k) & (df_total['r'] == r) & (df_total['case'] == case)]
    expected_size = n * r
    current_size = len(df_prov)
    if expected_size == current_size:
        print(f"Tamanho Esperado: {expected_size} | Tamanho Calculado: {current_size} | OK")
    else:
        print(f"Tamanho Esperado: {expected_size} | Tamanho Calculado: {current_size} | Mismatch")
    print()

Combinação N = 2; K = 2; R = 10; Case = 0
Tamanho Esperado: 20 | Tamanho Calculado: 20 | OK

Combinação N = 2; K = 2; R = 10; Case = 1
Tamanho Esperado: 20 | Tamanho Calculado: 20 | OK

Combinação N = 4; K = 2; R = 10; Case = 1
Tamanho Esperado: 40 | Tamanho Calculado: 60 | Mismatch



In [7]:
df_prov

,Thread ID,Time,n,r,k,case
20,2,10:12:41.216000,4,10,2,1
21,1,10:12:43.223000,4,10,2,1
22,2,10:12:45.232000,4,10,2,1
23,1,10:12:47.239000,4,10,2,1
24,2,10:12:49.248000,4,10,2,1
25,1,10:12:51.256000,4,10,2,1
26,2,10:12:53.265000,4,10,2,1
27,1,10:12:55.271000,4,10,2,1
28,2,10:12:57.280000,4,10,2,1
29,1,10:12:59.286000,4,10,2,1
